
# 📦 SLA Breach Prediction — Week 1  
## Business Understanding & Data Preprocessing

**Scope of Week 1**
- Understand the business problem (SLA breaches)
- Understand and trust the data
- Clean and validate the dataset
- Create the target variable (`sla_breach_flag`)

🚫 No visualization, feature engineering, or modeling in this phase.



## 1️⃣ Business Understanding

An **SLA (Service Level Agreement)** defines the expected delivery timeline agreed upon between the logistics provider and the client.

An **SLA breach** occurs when:
- The actual delivery time exceeds the planned delivery time.

### Why SLA Breach Prediction Matters
- Avoids contractual penalties
- Improves customer satisfaction
- Enables proactive intervention
- Supports vendor performance management


In [1]:

# 2️⃣ Load Dataset & Initial Assessment

import pandas as pd
import numpy as np

df = pd.read_csv('/content/b2b_sla_logistics_dataset_200k.csv')
df.head()


,shipment_id,carrier,shipping_mode,region,origin_country,destination_country,shipment_weight_kg,shipment_volume_cbm,priority_flag,fragile_flag,...,planned_delivery_days,actual_delivery_days,delivery_delay_days,sla_breach_flag,shipping_cost_usd,fuel_surcharge_pct,customs_delay_flag,weather_disruption_flag,shipment_value_usd,insurance_flag
0,SHP_1000000,Carrier_H,Rail,EMEA,France,UK,9256.28,100.28,0,0,...,23,24.0,1.0,1,9531.05,20.18,1,0,48235.35,0
1,SHP_1000001,Carrier_E,Rail,APAC,Vietnam,UK,11026.85,85.39,0,0,...,39,36.0,-3.0,0,6249.19,24.71,0,0,44935.84,1
2,SHP_1000002,Carrier_B,Road,AMER,Vietnam,Germany,9126.25,137.58,0,0,...,7,7.0,0.0,0,11949.56,12.06,0,0,90132.88,0
3,SHP_1000003,Carrier_G,Road,AMER,Mexico,USA,6327.51,51.24,0,0,...,18,15.0,-3.0,0,11227.29,23.35,0,0,84935.19,1
4,SHP_1000004,Carrier_D,Road,EMEA,Poland,China,8850.45,109.66,0,0,...,19,20.0,1.0,1,9746.15,11.44,0,0,62812.24,0


In [2]:

# Dataset shape and structure
df.shape, df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   shipment_id               200000 non-null  object 
 1   carrier                   200000 non-null  object 
 2   shipping_mode             200000 non-null  object 
 3   region                    200000 non-null  object 
 4   origin_country            200000 non-null  object 
 5   destination_country       200000 non-null  object 
 6   shipment_weight_kg        200000 non-null  float64
 7   shipment_volume_cbm       200000 non-null  float64
 8   priority_flag             200000 non-null  int64  
 9   fragile_flag              200000 non-null  int64  
 10  temperature_control_flag  200000 non-null  int64  
 11  planned_delivery_days     200000 non-null  int64  
 12  actual_delivery_days      200000 non-null  float64
 13  delivery_delay_days       200000 non-null  f

((200000, 21), None)

In [3]:

# Missing values summary
df.isnull().sum().sort_values(ascending=False)


,0
shipment_id,0
carrier,0
shipping_mode,0
region,0
origin_country,0
destination_country,0
shipment_weight_kg,0
shipment_volume_cbm,0
priority_flag,0
fragile_flag,0


In [4]:

# Duplicate records
df.duplicated().sum()


np.int64(0)


## Initial Findings (Assessment)
- Dataset size is sufficient for ML
- Missing values exist in cost, weight, and volume
- Duplicate records must be removed
- No structural corruption observed



## 3️⃣ Data Cleaning (Business-Justified)

### Strategy
- **Missing numerical values** → Median (robust to skewed logistics data)
- **Missing categorical values** → Mode (most frequent business scenario)
- **Duplicates** → Removed (duplicate shipments distort SLA metrics)


In [7]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('/content/b2b_sla_logistics_dataset_200k.csv')

# 1️⃣ Remove duplicates
df = df.drop_duplicates().reset_index(drop=True)

# 2️⃣ Identify column types
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# 3️⃣ Impute missing values (NO inplace warning)
df[num_cols] = df[num_cols].apply(lambda x: x.fillna(x.median()))
df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))

# 4️⃣ Final null check
df.isnull().sum().sum()



np.int64(0)


## 4️⃣ Target Variable Creation — SLA Breach

**Business Rule**
```
sla_breach_flag = 1  if actual_delivery_days > planned_delivery_days  
sla_breach_flag = 0  otherwise
```

This directly mirrors SLA contract terms.


In [8]:

# Target variable creation
df['sla_breach_flag'] = (df['actual_delivery_days'] > df['planned_delivery_days']).astype(int)

# Validate
df['sla_breach_flag'].value_counts(normalize=True)


,proportion
sla_breach_flag,
0,0.527645
1,0.472355



## 5️⃣ Encoding Categorical Variables

Prepared for future ML usage:
- Carrier
- Shipping mode
- Region
- Origin country
- Destination country

(No feature engineering performed)


In [12]:

from sklearn.preprocessing import LabelEncoder
import pandas as pd

encode_cols = [
    'carrier', 'shipping_mode', 'region',
    'origin_country', 'destination_country'
]

# Store encoders (important for future inverse transform / model use)
label_encoders = {}

print("🔹 Encoding categorical columns...\n")

for col in encode_cols:
    le = LabelEncoder()

    # Show unique values BEFORE encoding (sample)
    print(f"Column: {col}")
    print("Before encoding:", df[col].astype(str).unique()[:5])

    # Encode
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

    # Show unique values AFTER encoding (sample)
    print("After encoding:", df[col].unique()[:5])
    print("-" * 50)

print("\n✅ Encoding completed successfully.")
df[encode_cols].head()



🔹 Encoding categorical columns...

Column: carrier
Before encoding: ['7' '4' '1' '6' '3']
After encoding: [7 4 1 6 3]
--------------------------------------------------
Column: shipping_mode
Before encoding: ['1' '2' '0' '3']
After encoding: [1 2 0 3]
--------------------------------------------------
Column: region
Before encoding: ['2' '1' '0']
After encoding: [2 1 0]
--------------------------------------------------
Column: origin_country
Before encoding: ['1' '9' '4' '5' '7']
After encoding: [1 9 4 5 7]
--------------------------------------------------
Column: destination_country
Before encoding: ['7' '2' '8' '0' '9']
After encoding: [7 2 8 0 9]
--------------------------------------------------

✅ Encoding completed successfully.


,carrier,shipping_mode,region,origin_country,destination_country
0,7,1,2,1,7
1,4,1,1,9,7
2,1,2,0,9,2
3,6,2,0,4,8
4,3,2,2,5,0



## 6️⃣ Outlier Handling (Business-Aware)

Outliers reviewed for:
- Shipping cost
- Weight
- Volume
- Delivery delay

Approach:
- **Winsorization (1st–99th percentile)**
- Preserves extreme but valid business cases


In [13]:
# Winsorization with non-visual output (business-safe)

winsor_cols = [
    'shipping_cost_usd',
    'shipment_weight_kg',
    'shipment_volume_cbm',
    'delivery_delay_days'
]

print("🔹 Applying winsorization (1st–99th percentile)\n")

for col in winsor_cols:
    lower, upper = df[col].quantile([0.01, 0.99])

    # Capture stats BEFORE
    before_min = df[col].min()
    before_max = df[col].max()

    # Apply winsorization
    df[col] = df[col].clip(lower, upper)

    # Capture stats AFTER
    after_min = df[col].min()
    after_max = df[col].max()

    # Print non-visual output
    print(f"Column: {col}")
    print(f"  Before  → min: {before_min:.2f}, max: {before_max:.2f}")
    print(f"  After   → min: {after_min:.2f}, max: {after_max:.2f}")
    print("-" * 60)

print("\n✅ Winsorization completed successfully.")
df[winsor_cols].describe()

🔹 Applying winsorization (1st–99th percentile)

Column: shipping_cost_usd
  Before  → min: 545.59, max: 25345.78
  After   → min: 545.59, max: 25345.77
------------------------------------------------------------
Column: shipment_weight_kg
  Before  → min: 129.77, max: 11878.12
  After   → min: 129.77, max: 11878.12
------------------------------------------------------------
Column: shipment_volume_cbm
  Before  → min: 1.65, max: 148.55
  After   → min: 1.65, max: 148.55
------------------------------------------------------------
Column: delivery_delay_days
  Before  → min: -9.00, max: 10.00
  After   → min: -9.00, max: 10.00
------------------------------------------------------------

✅ Winsorization completed successfully.


,shipping_cost_usd,shipment_weight_kg,shipment_volume_cbm,delivery_delay_days
count,200000.000000,200000.000000,200000.000000,200000.000000
mean,6900.756747,6011.173820,75.041697,0.292055
std,4978.668748,3458.906380,43.264917,3.965727
min,545.589978,129.769996,1.650000,-9.000000
25%,3254.477500,3016.507500,37.480000,-2.000000
50%,5678.730000,6018.545000,75.055000,0.000000
75%,9252.337500,9003.557500,112.500000,3.000000
max,25345.770087,11878.120019,148.550000,10.000000



## 7️⃣ Final Data Validation Checklist ✅

- No negative costs
- Planned delivery days > 0
- Binary target valid
- Dataset ML-ready without leakage


In [15]:
assert (df['shipping_cost_usd'] >= 0).all()
assert (df['planned_delivery_days'] > 0).all()
assert set(df['sla_breach_flag'].unique()) == {0,1}

df.shape

(200000, 21)


## ✅ Week 1 Output

✔ Business-aligned, clean dataset  
✔ Target variable validated  
✔ No modeling bias introduced  
✔ Ready for Week 2 (EDA & Feature Engineering)
